<a href="https://colab.research.google.com/github/lyuoveta/git_test_model_1000/blob/master/BERT_1_version/BERT_1_clean(correct_incorrect).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install pytorch_lightning
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
from torchmetrics.functional import accuracy, auroc
from torchmetrics import F1Score
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import multiprocessing
multiprocessing.freeze_support()
import warnings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 18.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import random


df = pd.read_csv('IMDB_reviews_128_tag.csv')

In [ ]:
# deleting additional information
df.head()

,review,positive,negative
0,mani funni scene peopl dont normal pay attent ...,1,0
1,shame shame fine actor joseph fienn would allo...,0,1
2,movi pretti predict nuff saidfrom delay kiss s...,0,1
3,black mask jet li play bioengin superkil turn ...,1,0
4,watch entir movi recogn particip william hurt ...,0,1


In [ ]:
# split data
train_df, val_df = train_test_split(df, test_size=0.05)
print(train_df.shape, val_df.shape)

(1900, 3) (100, 3)


In [ ]:
LABEL_COLUMNS = df.columns.tolist()[1:]

In [ ]:
train_correct = train_df[train_df.iloc[:, 1] == 1]
train_incorrect = train_df[train_df.iloc[:, 2] == 1]

In [ ]:
# output of data quantity
count_ones = len(train_df[train_df.iloc[:, 1] == 1])
count_ones_2 = len(train_df[train_df.iloc[:, 2] == 1])
column_name = train_df.columns[1]
column_name_2 = train_df.columns[2]
print(column_name, count_ones, column_name_2, count_ones_2)

positive 960 negative 940


In [ ]:
train_df = pd.concat([
  train_incorrect,
  train_correct
])
print(train_df.shape, val_df.shape)

(1900, 3) (100, 3)


In [ ]:
print(train_df.columns)

Index(['review', 'positive', 'negative'], dtype='object')


In [ ]:
MAX_TOKEN_COUNT = 512

In [ ]:
# Dataset
warnings.filterwarnings("ignore", category=FutureWarning)


class CorrectReviewsDataset(Dataset):
    def __init__(
      self,
      data: pd.DataFrame,
      tokenizer: BertTokenizer,
      max_token_len: int = 128
    ):
      self.tokenizer = tokenizer
      self.data = data
      self.max_token_len = max_token_len
    def __len__(self):
      return len(self.data)
    def __getitem__(self, index: int):
      data_row = self.data.iloc[index]
      reviews = data_row.review
      labels = data_row[LABEL_COLUMNS]
      encoding = self.tokenizer.encode_plus(
        reviews,
        add_special_tokens=True,
        max_length=self.max_token_len,
        return_token_type_ids=False,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
        )
      return dict(
        reviews=reviews,
        input_ids=encoding["input_ids"].flatten(),
        attention_mask=encoding["attention_mask"].flatten(),
        #labels = torch.FloatTensor(labels.iloc[0]) #debug but tensor([])
        #labels = torch.FloatTensor(labels)
        labels=torch.FloatTensor(labels.iloc[:])
      )


In [ ]:
BERT_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# a sample item from the dataset
train_dataset = CorrectReviewsDataset(
  train_df,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)
sample_item = train_dataset[0]

In [ ]:
# Tokenization
BERT_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)
bert_model = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True).cuda()

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
train_dl = DataLoader(train_dataset, batch_size=8, num_workers=0)

In [ ]:
sample_batch = next(iter(train_dl))

In [ ]:
sample_batch["input_ids"]

tensor([[  101,  1107, 17030,  ...,     0,     0,     0],
        [  101,  1436,  1273,  ...,     0,     0,     0],
        [  101,  3008,  1612,  ...,     0,     0,     0],
        ...,
        [  101, 22572,  4626,  ...,     0,     0,     0],
        [  101,   189, 15243,  ...,     0,     0,     0],
        [  101,  1148,  1486,  ...,     0,     0,     0]])

In [ ]:
print(sample_batch["input_ids"].shape, sample_batch["attention_mask"].shape)

torch.Size([8, 512]) torch.Size([8, 512])


In [ ]:
# BERT
with torch.no_grad():
    output = bert_model(sample_batch["input_ids"].cuda(), sample_batch["attention_mask"].cuda())
    print(output.last_hidden_state.shape, output.pooler_output.shape)
    print(bert_model.config.hidden_size)

torch.Size([8, 512, 768]) torch.Size([8, 768])
768


In [ ]:
# custom dataset into a LightningDataModule
class CorrectReviewsDataModule(pl.LightningDataModule):
  def __init__(self, train_df, test_df, tokenizer, batch_size=8, max_token_len=128):
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.max_token_len = max_token_len
  def setup(self, stage=None):
    self.train_dataset = CorrectReviewsDataset(
      self.train_df,
      self.tokenizer,
      self.max_token_len
    )
    self.test_dataset = CorrectReviewsDataset(
      self.test_df,
      self.tokenizer,
      self.max_token_len
    )
  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size=self.batch_size,
      shuffle=True,
      num_workers=0
    )
  def val_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=0
    )
  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=0
    )

In [ ]:
# an instance of data module
N_EPOCHS = 10
BATCH_SIZE = 4
data_module = CorrectReviewsDataModule(
  train_df,
  val_df,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_TOKEN_COUNT
)

In [ ]:
# MODEL
class CorrectReviewsTagger(pl.LightningModule):
    def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.BCELoss()
    def forward(self, input_ids, attention_mask, labels=None):
        with torch.no_grad(): # не сохраняем градиенты
            output = self.bert(input_ids, attention_mask=attention_mask)
        output = self.classifier(output.pooler_output)
        output = torch.sigmoid(output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        return loss, output
    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": outputs, "labels": labels}
    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss
    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss
    def on_train_epoch_end(self):
        '''
        labels = []
        predictions = []
        for output in outputs:
            for out_labels in output["labels"].detach().cpu():
                labels.append(out_labels)
            for out_predictions in output["predictions"].detach().cpu():
                predictions.append(out_predictions)
        labels = torch.stack(labels).int()
        predictions = torch.stack(predictions)
        for i, name in enumerate(LABEL_COLUMNS):
            class_roc_auc = auroc(predictions[:, i], labels[:, i])
            self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", class_roc_auc, self.current_epoch)
        '''
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=2e-5)
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.n_warmup_steps,
            num_training_steps=self.n_training_steps
        )
        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
            scheduler=scheduler,
            interval='step'
            )
        )

In [ ]:
# Optimizer scheduler
dummy_model = nn.Linear(10, 1)
for param in dummy_model.parameters():
    param.requires_grad = True
optimizer = AdamW(params=dummy_model.parameters(), lr=0.001)
warmup_steps = 20
total_training_steps = 100
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=warmup_steps,
  num_training_steps=total_training_steps
)
learning_rate_history = []
for step in range(total_training_steps):
    optimizer.step()
    scheduler.step()
    learning_rate_history.append(optimizer.param_groups[0]['lr'])

In [ ]:
steps_per_epoch = len(train_df) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS
warmup_steps = total_training_steps // 5
print(warmup_steps, total_training_steps)

950 4750


In [ ]:
# an instance of the model
model: CorrectReviewsTagger = CorrectReviewsTagger(
  n_classes=len(LABEL_COLUMNS),
  n_warmup_steps=warmup_steps,
  n_training_steps=total_training_steps
)

In [ ]:
# Evaluation
criterion = nn.BCELoss()
prediction = torch.FloatTensor(
  [10.95873564, 1.07321467, 1.58524066, 0.03839076, 15.72987556, 1.09513213]
)
labels = torch.FloatTensor(
  [1., 0., 0., 0., 1., 0.]
)
print(torch.sigmoid(prediction))
print(criterion(torch.sigmoid(prediction), labels))

_, predictions = model(sample_batch["input_ids"], sample_batch["attention_mask"])
print(predictions)
print(criterion(predictions, sample_batch["labels"]))

tensor([1.0000, 0.7452, 0.8299, 0.5096, 1.0000, 0.7493])
tensor(0.8725)
tensor([[0.4653, 0.5590],
        [0.4537, 0.5585],
        [0.4633, 0.5425],
        [0.4684, 0.5486],
        [0.4563, 0.5505],
        [0.4551, 0.5545],
        [0.4568, 0.5532],
        [0.4755, 0.5350]], grad_fn=<SigmoidBackward0>)
tensor(0.6086, grad_fn=<BinaryCrossEntropyBackward0>)


In [ ]:
# Training

checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints",
  filename="best-checkpoint",
  save_top_k=1,
  verbose=True,
  monitor="val_loss",
  mode="min"
)

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="correct-reviews")

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
trainer = pl.Trainer(
  logger=logger,
  callbacks=[early_stopping_callback, checkpoint_callback],
  max_epochs=N_EPOCHS,
  accelerator="gpu",
  log_every_n_steps = 5
)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert       | BertModel | 108 M 
1 | classifier | Linear    | 1.5 K 
2 | criterion  | BCELoss   | 0     
-----------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.247   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 3800: 'val_loss' reached 0.68925 (best 0.68925), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 4275: 'val_loss' reached 0.68917 (best 0.68917), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 4750: 'val_loss' reached 0.68914 (best 0.68914), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
trainer.test(model, datamodule=data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.6891363263130188     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.6891363263130188}]

In [ ]:
# Predictions
trained_model = CorrectReviewsTagger.load_from_checkpoint(
  trainer.checkpoint_callback.best_model_path,
  n_classes=len(LABEL_COLUMNS)
)
trained_model.eval()
trained_model.freeze()

In [ ]:
# TEST
test_comment = "I have been taking rhodiola for over a year.  It works great for reducing general anxiety.  If I go longer than two days without it, I get an inner jitteriness and feelings of doom.  I have noticed no side effects."
encoding = tokenizer.encode_plus(
  test_comment,
  add_special_tokens=True,
  max_length=512,
  return_token_type_ids=False,
  padding="max_length",
  return_attention_mask=True,
  return_tensors='pt',
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_ids = encoding["input_ids"].to(device)
attention_mask = encoding["attention_mask"].to(device)

trained_model = trained_model.to(device)

_, test_prediction = trained_model(input_ids, attention_mask)
test_prediction = test_prediction.flatten().cpu().numpy()

for label, prediction in zip(LABEL_COLUMNS, test_prediction):
    print(f"{label}: {prediction}")

positive: 0.5288928151130676
negative: 0.46764224767684937


In [ ]:
# TEST 2
THRESHOLD = 0.5
test_comment = "as a nourishment for old dry skin, people can not believe I am 74 years old"
encoding = tokenizer.encode_plus(
  test_comment,
  add_special_tokens=True,
  max_length=512,
  return_token_type_ids=False,
  padding="max_length",
  return_attention_mask=True,
  return_tensors='pt',
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_ids = encoding["input_ids"].to(device)
attention_mask = encoding["attention_mask"].to(device)

trained_model = trained_model.to(device)

with torch.no_grad():
    _, test_prediction = trained_model(input_ids, attention_mask)

test_prediction = test_prediction.flatten().cpu().numpy()

for label, prediction in zip(LABEL_COLUMNS, test_prediction):
  if prediction < THRESHOLD:
    continue
  print(f"{label}: {prediction:.4f}")

positive: 0.5155


In [ ]:
# Evaluation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = trained_model.to(device)
val_dataset = CorrectReviewsDataset(
  val_df,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)
predictions = []
labels = []
for item in tqdm(val_dataset):
  _, prediction = trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device),
    item["attention_mask"].unsqueeze(dim=0).to(device)
  )
  predictions.append(prediction.flatten())
  labels.append(item["labels"].int())
predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
task = 'BINARY'
acc = accuracy(predictions, labels, task, threshold=0.5)
print(f"accuracy: {acc.item():.4f}")


accuracy: 0.5300


In [ ]:
#the ROC for each tag
print("AUROC per tag")
for i, name in enumerate(LABEL_COLUMNS):
  tag_auroc = roc_auc_score(labels[:, i], predictions[:, i])
  print(f"{name}: {tag_auroc}")

AUROC per tag
positive: 0.531812725090036
negative: 0.5654261704681873


In [ ]:
y_pred = predictions.numpy()
y_true = labels.numpy()
upper, lower = 1, 0
y_pred = np.where(y_pred > THRESHOLD, upper, lower)
print(classification_report(
  y_true,
  y_pred,
  target_names=LABEL_COLUMNS,
  zero_division=0
))


              precision    recall  f1-score   support

    positive       0.54      0.75      0.62        51
    negative       0.51      0.41      0.45        49

   micro avg       0.53      0.58      0.55       100
   macro avg       0.52      0.58      0.54       100
weighted avg       0.52      0.58      0.54       100
 samples avg       0.53      0.58      0.54       100



In [ ]:
print(metrics.f1_score(y_true, y_pred,average='micro'))

0.5523809523809523
